In [28]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from tqdm import tqdm

Let's fetch the dataframes.

In [16]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
vtt_domain_df = pd.read_csv("./data/dataframes/df_relationships_vtt_domain.csv")
comp_domain_df = pd.read_csv("./data/dataframes/df_relationships_comp_domain.csv")

In [24]:
# Clean the VTT domain df.

vtt_domain_df["text_to_compare"] = (
    vtt_domain_df["source id"].fillna("") + " " +
    vtt_domain_df["source english_id"].fillna("") + " " +
    vtt_domain_df["source description"].fillna(""))

vtt_domain_df = vtt_domain_df[vtt_domain_df["source type"] != "Organization"]
vtt_domain_df = vtt_domain_df.drop_duplicates(subset="source description", keep="first")

# Clean the comp domain df.

comp_domain_df["text_to_compare"] = (
    comp_domain_df["source id"].fillna("") + " " +
    comp_domain_df["source description"].fillna(""))

comp_domain_df = comp_domain_df[comp_domain_df["source type"] != "Organization"]
comp_domain_df = comp_domain_df.drop_duplicates(subset="source description", keep="first")

print("Comp domain df length:", len(comp_domain_df), " - VTT domain df length:", len(vtt_domain_df))

Comp domain df length: 1528  - VTT domain df length: 1504


Embed everything.

In [25]:
vtt_embeddings = model.encode(vtt_domain_df["text_to_compare"].tolist(), convert_to_tensor=True)
comp_embeddings = model.encode(comp_domain_df["text_to_compare"].tolist(), convert_to_tensor=True)

In [33]:
threshold = 0.80
similar_pairs = []

print("Comparing entries pair by pair...")

for i in tqdm(range(len(vtt_embeddings))):
    for j in range(i + 1, len(comp_embeddings)):
        sim = util.cos_sim(vtt_embeddings[i], comp_embeddings[j]).item()
        
        if sim > threshold:
            source_i = vtt_domain_df.iloc[i]["text_to_compare"]
            source_j = comp_domain_df.iloc[j]["text_to_compare"]
            
            print(f"\nSimilarity: {sim:.3f} | From: {i+1} To: {j+1}")
            print(f"→ Source {i+1} description: {source_i}")
            print(f"→ Source {j+1} description: {source_j}")
            
            similar_pairs.append((i, j, sim))

# Output results
print(f"\nFound {len(similar_pairs)} high-similarity pairs (> {threshold}):")
for i, j, score in similar_pairs:
    print(f"[{i}] '{df.iloc[i]['source english_id']}' <--> [{j}] '{df.iloc[j]['source english_id']}' | Similarity: {score:.3f}")

Comparing entries pair by pair...


  2%|▏         | 31/1504 [00:24<19:50,  1.24it/s]


Similarity: 0.859 | From: 32 To: 884
→ Source 32 description: 5G-Safe project 5G-Safe project A project exploring the use of the 5G mobile network to improve road safety through fast data transmission solutions, collecting vehicle and road data for applications such as road weather services, road maintenance, and self-driving car control.
→ Source 884 description: 5G-Safe -hanke The 5G-Safe project developing new vehicle network solutions and local road weather and safety services to support drivers, road maintainers, and autonomous vehicle control systems by utilizing 5G technology for real-time data services.


  5%|▍         | 72/1504 [00:55<17:38,  1.35it/s]


Similarity: 0.865 | From: 73 To: 497
→ Source 73 description: Ioncell-F technology Ioncell-F technology A technology based on direct dissolution of cellulose using ionic liquid solvent to produce strong textile fibres by dry-jet wet spinning.
→ Source 497 description: Ioncell-F technology A technology based on direct dissolution of cellulose using dry-jet wet spinning technique, developed in collaboration between Aalto University and University of Helsinki.


...


Similarity: 0.922 | From: 770 To: 814
→ Source 770 description: quantum encryption test network in Finland quantum encryption test network in Finland A public test network built in the Helsinki Metropolitan Area enabling the introduction of quantum encryption technologies, allowing Finnish companies to develop and test services and products based on quantum encryption.
→ Source 814 description: Quantum encryption test network in Finland A quantum encryption test network built in Finland to strengthen the cybersecurity infrastructure as part of an EU-wide system, implemented with commercial devices and completed in the first half of 2024.


 51%|█████     | 770/1504 [08:36<05:12,  2.35it/s]


Similarity: 0.944 | From: 770 To: 1522
→ Source 770 description: quantum encryption test network in Finland quantum encryption test network in Finland A public test network built in the Helsinki Metropolitan Area enabling the introduction of quantum encryption technologies, allowing Finnish companies to develop and test services and products based on quantum encryption.
→ Source 1522 description: Quantum encryption test network built in Finland A test network using quantum encryption technology constructed in Finland as part of the EU-wide quantum encryption initiative to strengthen cybersecurity infrastructure.


 53%|█████▎    | 801/1504 [08:49<07:44,  1.51it/s]


KeyboardInterrupt: 